In [ ]:
import nibabel as nib

image_path = r'[path]'
seg_path = r'[path]'

img = nib.load(image_path)
seg = nib.load(seg_path)

print("Image Spacing:", img.header.get_zooms())
print("Segmentation Spacing:", seg.header.get_zooms())


In [ ]:
# Print header information
print("NIfTI Image Orientation:\n", nib.aff2axcodes(img.affine))
print("Label NIfTI Orientation:\n", nib.aff2axcodes(seg.affine))

### Matching Space

In [ ]:
import os
import nibabel as nib
import numpy as np

def match_spacing(input_image_folder, input_label_folder, output_image_folder):

    os.makedirs(output_image_folder, exist_ok=True)

    for image_file in os.listdir(input_image_folder):
        if image_file.endswith(".nii.gz"):
            image_path = os.path.join(input_image_folder, image_file)
            label_path = os.path.join(input_label_folder, image_file.replace("_0000.nii.gz", ".nii.gz")) 
            output_image_path = os.path.join(output_image_folder, image_file)

            if not os.path.exists(label_path):
                print(f"Label file not found for image: {image_file}. Skipping...")
                continue

            image = nib.load(image_path)
            label = nib.load(label_path)

            label_affine = label.affine
            label_spacing = np.sqrt(np.sum(label_affine[:3, :3] ** 2, axis=0))

            image_affine = image.affine.copy()
            image_spacing = np.sqrt(np.sum(image_affine[:3, :3] ** 2, axis=0))

            for i in range(3):
                scaling_factor = label_spacing[i] / image_spacing[i]
                image_affine[:3, i] *= scaling_factor

            corrected_image = nib.Nifti1Image(image.get_fdata(), image_affine, header=image.header)
            nib.save(corrected_image, output_image_path)

            print(f"Corrected image saved to: {output_image_path}")

input_image_folder = r"[path]"
input_label_folder = r"[path]"
output_image_folder = r"[path]"

match_spacing(input_image_folder, input_label_folder, output_image_folder)


### Matching Origin

In [ ]:
import os
import nibabel as nib
import numpy as np

def match_origin(input_image_folder, input_label_folder, output_image_folder):

    os.makedirs(output_image_folder, exist_ok=True)


    for image_file in os.listdir(input_image_folder):
        if image_file.endswith(".nii.gz"):
            image_path = os.path.join(input_image_folder, image_file)
            label_path = os.path.join(input_label_folder, image_file.replace("_0000.nii.gz", ".nii.gz")) 
            output_image_path = os.path.join(output_image_folder, image_file)

            if not os.path.exists(label_path):
                print(f"Label file not found for image: {image_file}. Skipping...")
                continue

            image = nib.load(image_path)
            label = nib.load(label_path)

            # Get and compare origins
            image_origin = image.affine[:3, 3]
            label_origin = label.affine[:3, 3]

            if not np.allclose(image_origin, label_origin):
                print(f"Origin mismatch for {image_file}:")
                print(f"  Image origin: {image_origin}")
                print(f"  Label origin: {label_origin}")

                corrected_label_affine = label.affine.copy()
                corrected_label_affine[:3, 3] = image_origin

                corrected_label = nib.Nifti1Image(label.get_fdata(), corrected_label_affine, header=label.header)
                nib.save(corrected_label, label_path)
                print(f"  Corrected label origin saved for {image_file}")

            else:
                print(f"Origins match for {image_file}. No correction needed.")

input_image_folder = r"[path]"
input_label_folder = r"[Path]"
output_image_folder = r"[path]"

match_origin(input_image_folder, input_label_folder, output_image_folder)